## Agency Grain Metrics
* Add it to the pipeline in `rt_scheduled_v_ran/scripts/rt_v_scheduled_agency.py`
* `cd data-analyses/rt_segment_speeds && pip install -r requirements.txt && cd ../_shared_utils && make setup_env && cd ../gtfs_digest`

In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
from segment_speed_utils import (
    gtfs_schedule_wrangling,
    helpers,
    metrics,
    time_series_utils,
)
from segment_speed_utils.project_vars import (
    COMPILED_CACHED_VIEWS,
    GTFS_DATA_DICT,
    PROJECT_CRS,
    RT_SCHED_GCS,
    SCHED_GCS,
    SEGMENT_GCS,
)
from shared_utils import catalog_utils, rt_dates, rt_utils

In [2]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

### Exploring
* Need to filer for only ("sched_rt_category", "==", "schedule_and_vp")

In [3]:
dict_inputs = GTFS_DATA_DICT.rt_vs_schedule_tables

In [4]:
rt_dates.y2024_dates

['2024-01-17',
 '2024-02-14',
 '2024-03-13',
 '2024-04-17',
 '2024-05-22',
 '2024-06-12',
 '2024-07-17',
 '2024-08-14',
 '2024-09-18']

In [5]:
RT_SCHED_GCS

'gs://calitp-analytics-data/data-analyses/rt_vs_schedule/'

In [6]:
ROUTE_EXPORT = dict_inputs.vp_route_direction_metrics

In [7]:
ROUTE_EXPORT

'vp_route_dir/route_direction_metrics'

In [8]:
analysis_date = "2024-09-18"

In [9]:
df = pd.read_parquet(f"{RT_SCHED_GCS}{ROUTE_EXPORT}_{analysis_date}.parquet")

In [10]:
df = df.loc[df["time_period"] == "all_day"].reset_index(drop=True)

In [34]:
df.sample().T

,1506
schedule_gtfs_dataset_key,91af7482fde58c6261f386b732404e11
route_id,155
direction_id,1.00
time_period,all_day
minutes_atleast1_vp,362
minutes_atleast2_vp,353
total_rt_service_minutes,348.00
total_scheduled_service_minutes,455.00
total_vp,1056
vp_in_shape,1053


In [11]:
groupby_cols = [
    "caltrans_district",
    "organization_name",
    "schedule_gtfs_dataset_key",
]

In [12]:
agg1 = (
    df.groupby(groupby_cols)
    .agg(
        {
            "total_vp": "sum",
            "vp_in_shape": "sum",
            "total_rt_service_minutes": "sum",
        }
    )
    .reset_index()
)

In [13]:
agg1["vp_per_min_agency"] = ((agg1.total_vp / agg1.total_rt_service_minutes)).round(2)
agg1["spatial_accuracy_agency"] = ((agg1.vp_in_shape / agg1.total_vp) * 100).round(2)

In [14]:
63718 / 97768

0.6517265362899927

In [15]:
agg1.loc[agg1.organization_name == "Marin County Transit District"].T

,23
caltrans_district,04 - Oakland
organization_name,Marin County Transit District
schedule_gtfs_dataset_key,015d67d5b75b5cf2b710bbadadfb75f5
total_vp,97768
vp_in_shape,63718
total_rt_service_minutes,36831.13
vp_per_min_agency,2.65
spatial_accuracy_agency,65.17


### Original

In [16]:
organization_name = "Marin County Transit District"

In [17]:
schd_vp_url = f"{GTFS_DATA_DICT.digest_tables.dir}{GTFS_DATA_DICT.digest_tables.route_schedule_vp}.parquet"

# Keep only rows that are found in both schedule and real time data
vp_sched_df = pd.read_parquet(schd_vp_url)

In [18]:
vp_sched_df = vp_sched_df.loc[vp_sched_df["time_period"] == "all_day"].reset_index(
    drop=True
)

In [26]:
vp_sched_df = vp_sched_df.loc[vp_sched_df.sched_rt_category == "schedule_and_vp"]

In [27]:
vp_sched_df2 = (
    vp_sched_df.groupby(
        [
            "caltrans_district",
            "organization_name",
            "schedule_gtfs_dataset_key",
            "service_date",
        ]
    )
    .agg(
        {
            "total_vp": "sum",
            "total_rt_service_minutes": "sum",
            "vp_in_shape": "sum",
        }
    )
    .reset_index()
)

In [28]:
vp_sched_df2.loc[
    (vp_sched_df2.organization_name == organization_name)
    & (vp_sched_df2.service_date == "2024-09-18")
]

,caltrans_district,organization_name,schedule_gtfs_dataset_key,service_date,total_vp,total_rt_service_minutes,vp_in_shape
476,04 - Oakland,Marin County Transit District,015d67d5b75b5cf2b710bbadadfb75f5,2024-09-18,67420,25282.30,61736


In [29]:
vp_sched_df3 = (
    vp_sched_df.groupby(["caltrans_district", "organization_name", "service_date"])
    .agg(
        {
            "total_vp": "sum",
            "total_rt_service_minutes": "sum",
            "vp_in_shape": "sum",
        }
    )
    .reset_index()
)

In [30]:
vp_sched_df3.loc[
    (vp_sched_df3.organization_name == organization_name)
    & (vp_sched_df3.service_date == "2024-09-18")
]

,caltrans_district,organization_name,service_date,total_vp,total_rt_service_minutes,vp_in_shape
477,04 - Oakland,Marin County Transit District,2024-09-18,67420,25282.30,61736


In [31]:
63718 / 97768

0.6517265362899927